In [104]:
import requests
from bs4 import BeautifulSoup
import re # a library to do regular expression matching
import pandas as pd
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
import numpy as np
import lxml

In [114]:
import warnings
warnings.filterwarnings('ignore')

In [203]:
page = requests.get("https://www.top100startups.swiss/index.cfm?CFID=72313628&CFTOKEN=b25424f255d2862f-57043D28-C7D9-9B77-739FB33ACE36FC5F&page=136374", timeout=15)
print(page)
print(page.text[:500])

<Response [200]>

<!DOCTYPE html>
<html lang="en">
<head>
	<meta charset="UTF-8">
		
		<title>TOP 100 Swiss Startup Award Ranking 2011 - top100startup.ch</title>
		
		<meta name="Description" content="TOP 100 Swiss Startup Award Ranking 2011: The best Swiss Startups 2011 picked by a panel of 100 leading investors and startup experts." />				
	
		<meta name="Keywords" content="TOP 100, TOP 100 Swiss Startup Award, Best Swiss Startups, Venturelab, Swiss Startups, Innovation, Startups, TOP100SSU, Startup 


In [204]:
soup= BeautifulSoup(page.content, "html.parser")

In [205]:
soup2=soup.findAll("ul", {"class":"table-top100"})[0]

In [206]:
#Rank
soup2.findAll("span", {"class":"col-rank"})[1].text.split("Rank")[-1]  #here we have "Ranke" + "1"

'1'

In [207]:
#Name
soup2.findAll("span", {"class":"col-star","class":"text"})[0].text

'Optotune AG'

In [208]:
#Headquarter
soup2.findAll("span", {"class":"col-head"})[1].text.split("Headquarter")[-1]  #here we have "Headquarter" + "Zurich"

'Dietikon'

In [214]:
#Category
soup2.findAll("span", {"class":"col-cate"})[98].text.split("Category")[-1]  #here we have "Category" + "Fintech"

'Medtech'

In [194]:
#Description
soup2.findAll("span", {"class":"col-desc"})[1].text.strip().split("Description\n\n\r\n\t\t\t\t\t")[-1]

'Online marketplace for holiday rentals.'

In [195]:
#Link
soup2.findAll("a")[0].get("href")

'/HouseTrip'

In [115]:
def get_data(year,URL):  #get data for 2022
    page = requests.get(URL, timeout=15)
    soup= BeautifulSoup(page.content, "html.parser")
    soup2=soup.findAll("ul", {"class":"table-top100"})[0]
    
    cols = ["rank","name","headquarter","category","description","link","year"]
    df = pd.DataFrame(columns = cols)
    
    for i in range(100):
        rank = soup2.findAll("span", {"class":"col-rank"})[i+1].text.split("Rank")[-1]
        name = soup2.findAll("span", {"class":"col-star","class":"text"})[i].text
        headquarter = soup2.findAll("span", {"class":"col-head"})[i+1].text.split("Headquarter")[-1] 
        category = soup2.findAll("span", {"class":"col-cate"})[i+1].text.split("Category")[-1]
        description = soup2.findAll("span", {"class":"col-desc"})[i+1].text.strip().split("  ")[-1].strip(" \t")
        link = "https://www.top100startups.swiss"+ soup2.findAll("a")[i].get("href")
        year = year
        
        dict = { 
        'rank': rank,
        "name":name,
        "headquarter":headquarter,
        "category":category,
        "description":description,
        "link":link,
        "year":year}
        
        df = df.append(dict, ignore_index=True)
    return df

In [220]:
def get_data2(year,URL):  #get data for and before 2021 
    page = requests.get(URL, timeout=15)
    soup= BeautifulSoup(page.content, "html.parser")
    soup2=soup.findAll("ul", {"class":"table-top100"})[0]
    
    cols = ["rank","name","headquarter","category","description","link","year"]
    df = pd.DataFrame(columns = cols)
    
    for i in range(98): #99 for 2012, 98 for 2011
        rank = soup2.findAll("span", {"class":"col-rank"})[i+1].text.split("Rank")[-1]
        name = soup2.findAll("span", {"class":"col-star","class":"text"})[i].text
        headquarter = soup2.findAll("span", {"class":"col-head"})[i+1].text.split("Headquarter")[-1] 
        category = soup2.findAll("span", {"class":"col-cate"})[i+1].text.split("Category")[-1]
        description = soup2.findAll("span", {"class":"col-desc"})[i+1].text.strip().split("Description\n\n\r\n\t\t\t\t\t")[-1]
        link = "https://www.top100startups.swiss"+ soup2.findAll("a")[i].get("href")
        year = year
        
        dict = { 
        'rank': rank,
        "name":name,
        "headquarter":headquarter,
        "category":category,
        "description":description,
        "link":link,
        "year":year}
        
        df = df.append(dict, ignore_index=True)
    return df

In [216]:
df=get_data2(2011,"https://www.top100startups.swiss/index.cfm?CFID=72313628&CFTOKEN=b25424f255d2862f-57043D28-C7D9-9B77-739FB33ACE36FC5F&page=136374")

In [219]:
df.to_csv("2011_top_98_startups.csv")

In [249]:
df.reset_index()

rank                                           name         headquarter  \
0     1                                    Optotune AG            Dietikon   
1     2                                     Dacuda AG               Zürich   
2     3                                     Doodle AG               Zürich   
3     4                                    Lemoptix SA            Lausanne   
4     5                     Poken SA (acquired by GES)            Lausanne   
5     6                                    InSphero AG           Schlieren   
6     7                    Finity SA (former paper.li)            Lausanne   
7     8                                  HouseTrip SA               London   
8     9                                    Malcisbo AG           Schlieren   
9    10               Aïmago SA  (acquired by Novadaq)            Lausanne   
10   11                                  Climeworks AG              Zürich   
11   12                                   Biognosys AG  Zürich - Schlieren   
12   13                                   Redbiotec AG           Schlieren   
13   14                                  Neurimmune AG           Schlieren   
14   15                      Lightbend Sàrl (Typesafe)            Lausanne   
15   16                                     Kooaba AG            Cupertino   
16   17                                   Celeroton AG          Volketswil   
17   18                                GetYourGuide AG              Zürich   
18   19                     Stemergie Biotechnology SA              Geneva   
19   20                                    greenTEG AG             Rümlang   
20   21                    Atizo AG (acquired by Hyve)                Bern   
21   22            LiberoVision AG (acquired by Vizrt)              Zürich   
22   23                                   BioVersys AG               Basel   
23   24                 Arktis Radiation Detectors Ltd              Zürich   
24   25                                   Biocartis SA            Lausanne   
25   26                                  QualySense AG          Glattbrugg   
26   27             Procedural (acquired by Esri Ltd.)            Redlands   
27   28                         PRECLIN Biosystems SA            Epalinges   
28   29  Cilag GmbH International (former Covagen AG)     Zürich-Schlieren   
29   30                                    VirtaMed AG           Schlieren   
30   31                         Optical Additives GmbH             Staufen   
31   32                                     Abionic SA           Epalinges   
32   33                                   Credentis AG            Windisch   
33   34                                      ViSSee AG                 Zug   
34   35                                     restorm AG              Zürich   
35   36         Andiast Technolgoies GmbH (smartie.ch)              Zürich   
36   37                                 CAScination AG                Bern   
37   38                      BioXpress Therapeutics SA     Plan-les-Quates   
38   39                          Zurich Instruments AG              Zürich   
39   40                                     Squirro AG              Zürich   
40   41                                   Attolight SA            Lausanne   
41   42                                  Kandou Bus SA            Lausanne   
42   43             Goodshine AG (acquired by Ringier)              Zürich   
43   44                                        joiz AG              Zürich   
44   45                                 GenKyoTex S.A.     Plan-Les-Ouates   
45   47                                       YouRehab           Schlieren   
46   48                                     DECTRIS AG    Baden - Daettwil   
47   49                                   Swisstom AG            Landquart   
48   50     Luciole Medical AG (former NeMoDevices AG)              Zürich   
49   51                                         Limmex              Zürich   
50   52                  

In [226]:
for y in range(2011,2023):
    print(y)
    df_y=pd.read_csv("../data/"+(str(y)+"_top_100_startups.csv"))

In [238]:
df_2011 = pd.read_csv("../data/2011_top_100_startups.csv")
df_2012 = pd.read_csv("../data/2012_top_100_startups.csv")
df_2013 = pd.read_csv("../data/2013_top_100_startups.csv")
df_2014 = pd.read_csv("../data/2014_top_100_startups.csv")
df_2015 = pd.read_csv("../data/2015_top_100_startups.csv")
df_2016 = pd.read_csv("../data/2016_top_100_startups.csv")
df_2017 = pd.read_csv("../data/2017_top_100_startups.csv")
df_2018 = pd.read_csv("../data/2018_top_100_startups.csv")
df_2019 = pd.read_csv("../data/2019_top_100_startups.csv")
df_2020 = pd.read_csv("../data/2020_top_100_startups.csv")
df_2021 = pd.read_csv("../data/2021_top_100_startups.csv")
df_2022 = pd.read_csv("../data/2022_top_100_startups.csv")

In [239]:
df_list=[df_2022,df_2021,df_2020,df_2019,df_2018,df_2017,df_2016,df_2015,df_2014,df_2013,df_2012,df_2011]

In [246]:
big_df

index  Unnamed: 0  rank  \
0         0           0     1   
1         1           1     2   
2         2           2     3   
3         3           3     4   
4         4           4     5   
5         5           5     6   
6         6           6     7   
7         7           7     8   
8         8           8     9   
9         9           9    10   
10       10          10    11   
11       11          11    12   
12       12          12    13   
13       13          13    14   
14       14          14    15   
15       15          15    16   
16       16          16    17   
17       17          17    18   
18       18          18    19   
19       19          19    20   
20       20          20    21   
21       21          21    22   
22       22          22    23   
23       23          23    24   
24       24          24    25   
25       25          25    26   
26       26          26    27   
27       27          27    28   
28       28          28    29   
29       29          29    30   
30       30          30    31   
31       31          31    32   
32       32          32    33   
33       33          33    34   
34       34          34    35   
35       35          35    36   
36       36          36    37   
37       37          37    38   
38       38          38    39   
39       39          39    40   
40       40          40    41   
41       41          41    42   
42       42          42    43   
43       43          43    44   
44       44          44    45   
45       45          45    46   
46       46          46    47   
47       47          47    48   
48       48          48    49   
49       49          49    50   
50       50          50    51   
51       51          51    52   
52       52          52    53   
53       53          53    54   
54       54          54    55   
55       55          55    56   
56       56          56    57   
57       57          57    58   
58       58          58    59   
59       59          59    60   
60       60          60    61   
61       61          61    62   
62       62          62    63   
63       63          63    64   
64       64          64    65   
65       65          65    66   
66       66          66    67   
67       67          67    68   
68       68          68    69   
69       69          69    70   
70       70          70    71   
71       71          71    72   
72       72          72    73   
73       73          73    74   
74       74          74    75   
75       75          75    76   
76       76          76    77   
77       77          77    78   
78       78          78    79   
79       79          79    80   
80       80          80    81   
81       81          81    82   
82       82          82    83   
83       83          83    84   
84       84          84    85   
85       85          85    86   
86       86          86    87   
87       87          87    88   
88       88          88    89   
89       89          89    90   
90       90          90    91   
91       91          91    92   
92       92          92    93   
93       93          93    94   
94       94          94    95   
95       95          95    96   
96       96          96    97   
97       97          97    98   
98       98          98    99   
99       99          99   100   
100       0           0     1   
101       1           1     2   
102       2           2     3   
103       3           3     4   
104       4           4     5   
105       5           5     6   
106       6           6     7   
107       7           7     8   
108       8           8     9   
109       9           9    10   
110      10          10    11   
111      11          11    12   
112      12          12    13   
113      13          13    14   
114      14          14    15   
115      15          15    16   
116      16          16    17   
117      17          17    18   
118      18          18    19   
119      19          19    20   
120      20  

In [252]:
#big_df.to_csv("all_top_100_startups.csv")

In [264]:
big_df=big_df.drop(["index","Unnamed: 0"],axis=1)

In [268]:
big_df.head(2)

rank              name headquarter  category  \
0     1    Yokoy Group AG      Zurich   Fintech   
1     2  Planted Foods AG   Kemptthal  Foodtech   

                                         description  \
0  Your expenses and company credit cards on auto...   
1                                      Planted meats   

                                       link  year  
0    https://www.top100startups.swiss/yokoy  2022  
1  https://www.top100startups.swiss/Planted  2022

In [269]:
big_df.groupby(["category","year"]).name.count()

category     year
Biotech      2011    17
             2012    16
             2013    13
             2014    17
             2015    15
             2016    19
             2017    24
             2018    17
             2019    18
             2020    21
             2021    17
             2022    19
Cleantech    2011     5
             2012    10
             2013    12
             2014     9
             2015     8
             2016     8
             2017     7
             2018     6
             2019     4
             2020     4
             2021     6
             2022     9
Engineering  2011    19
             2012    21
             2013    21
             2014    16
             2015    15
             2016     9
             2017     7
             2018    12
             2019    10
             2020     7
             2021    12
             2022     9
Fintech      2011     2
             2012     1
             2013     1
             2014     1
             2015     

In [271]:
big_df.head(4)

rank              name headquarter  category  \
0     1    Yokoy Group AG      Zurich   Fintech   
1     2  Planted Foods AG   Kemptthal  Foodtech   
2     3          Ledgy AG      Zurich   Fintech   
3     4         CUTISS AG   Schlieren   Biotech   

                                         description  \
0  Your expenses and company credit cards on auto...   
1                                      Planted meats   
2                         Your equity set for growth   
3                                  Personalized skin   

                                       link  year  
0    https://www.top100startups.swiss/yokoy  2022  
1  https://www.top100startups.swiss/Planted  2022  
2    https://www.top100startups.swiss/Ledgy  2022  
3   https://www.top100startups.swiss/CUTISS  2022

In [272]:
test = pd.read_excel ("../data/new_found_companies_by_canton.xlsx")

ImportError: Missing optional dependency 'openpyxl'.  Use pip or conda to install openpyxl.